# Step 1: Install and Import dependencies

In [ ]:
!pip install -U --quiet langchain langchain_community chromadb  langchain-google-vertexai
!pip install --quiet "unstructured[all-docs]" pypdf pillow pydantic lxml pillow matplotlib chromadb tiktoken
!pip install --upgrade numpy
!pip install --upgrade langchain_google_vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 67.5 MB/s eta 0:00

In this we are specifying the project ID of GCP, and credentials that you can request either Shubham or Puneet.

In [ ]:
import os
PROJECT_ID = "rag-demo-454104"
REGION = "asia-south1"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/rag-demo-454104-9fdb4300f7d3.json"

In [ ]:
import vertexai
vertexai.init(project = PROJECT_ID , location = REGION) # This line is used to initialize the Vertex AI API for the given project and location

# Step 2: Prepare and load data

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader2 = PyPDFLoader("/content/Student_Handbook.pdf")
docs2 = loader2.load()

docs = docs2

texts = [d.page_content for d in docs]

tables = [] # QUESTION: Where is the code to process tables that are there in the PDF?

In [ ]:
len(texts) # Meaning there are total 36 pages in the PDF docs that we uploaded

36

In [ ]:
texts[5] # This is the actual content of the sixth page

'4.   Fees for the entireprogramme\n● First year fees kept low to enable learners to try out theprogramme\n● Entry fee for Qualifier exam: Rs. 3000 (non-refundable)- with suitable waivers\n● Fee waivers for learners belonging to certain categoriesand economic backgrounds\nNumber of credits in each level:\nLevel Theory Project Apprenticeship NPTEL MG/HS/HM\nFoundation 32\nDiploma inDS 23 4\nDiploma inP 23 4\nBSc 28-X X (0-4)\nBS 28-A-B A (0/4/12) B=4\n*Only maximum of 24 credits can be transferredtowards non counted CGPA course\nFoundation: Rs 32000/-Diploma Level: 62500 x 2 = Rs 125,000/-BSc Level: Rs 2.21L - 2.47L        BS Level: 3.25L - 3.87L\nCostpercredit(Rs)\nNumber ofcredits Total fee Levelfees\nFoundation 1000/c 32c 32000 32,000\nDiploma 2500/cforTheory1250/cforproject\n23x2=46c4x2 = 8c 115,00010,000 125,000\nBSc Level 3- Rs2500/cLevel 4- Rs5000/c1000/cforNPTEL\n20c for 5mandatorycourses(Level 3)4cNPTEL+4cLevel 34c NPTEL+ 4c Level48c Level 34c Level 3+ 4c Level48c Level 4\n5000

In [ ]:
len(tables)

0

# Step 3: Generate Text Summaries

In [ ]:
from langchain_google_vertexai import VertexAI, ChatVertexAI, VertexAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [ ]:
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """
    prompt_text = """
    You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element}
    """

    prompt = PromptTemplate.from_template(prompt_text)

    empty_response = RunnableLambda(
        lambda x: AIMessage(content="Error processing document")
    )

    model_name = "gemini-pro"
    model = VertexAI(
        temperature=0, model_name=model_name, max_output_tokens=1024
    ).with_fallbacks([empty_response])

    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    text_summaries = []
    table_summaries = []

    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 1})
    elif texts:
        text_summaries = texts

    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 1})

    return text_summaries, table_summaries

In [ ]:
text_summaries, table_summaries = generate_text_summaries(
    texts, tables, summarize_texts=False
) # For 10 page PDF, it takes 37 seconds and 2 WARNINGS; 4m37s for full 36 page PDF; and also didn't use any money
# When summarize_texts=False, the code runs instantly..cool!

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# import pickle
# with open('/content/drive/My Drive/summaries.pkl', 'wb') as f:
#     pickle.dump([text_summaries, table_summaries], f)

# import pickle # Load later
# with open('/content/drive/My Drive/summaries.pkl', 'rb') as f:
#     text_summaries, table_summaries = pickle.load(f)

In [ ]:
from pprint import pprint
(text_summaries[0])

'Student Handbook\n        IITM BS Degree Office, 3rd Floor, ICSR Building, IIT Madras, Chennai - 600036                             support@study.iitm.ac.in\nPlease read this\nDear student,\nThis Student Handbook is intended to provide IIT MadrasBS Degree students with the information and policies theyshould be aware of, which may help them make the mostof the opportunities offered in this programme. It also givesyou formal notification and explanation of the programme’sregulations, policies and procedures.\nIt is essential, and your responsibility, to read it andfamiliarize yourself with the content. It should be kepthandy and referred to during your time with us.\nThis handbook may be revised time and again. While thisstudent handbook was prepared on the basis of the bestinformation available at the time of publication, allinformation, including policies and procedures are subjectto change without notice or obligation.\nAll rules and regulations in this document are approved byour c

In [ ]:
table_summaries

[]

# Step 4: Generate Image Summaries

In [ ]:
import base64
import os
from langchain_core.messages import HumanMessage
from langchain_google_vertexai import ChatVertexAI

In [ ]:
def encode_image(image_path): # This function encodes the image to base64 string
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Generates a textual summary of an image using the ChatVertexAI model with the 'gemini-pro-vision' capability."""
    model = ChatVertexAI(model_name="gemini-pro-vision", max_output_tokens=1024)

    msg = model(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    img_base64_list = []

    image_summaries = []

    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""

    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries

img_base64_list, image_summaries = generate_img_summaries("/content/")

print(f"Number of image summaries: {len(image_summaries)}")

<ipython-input-15-a9c60bb58a06>:11: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  msg = model(


Number of image summaries: 3


In [ ]:
print(f"Summary 1: {image_summaries[0]}")
print(f"Summary 2: {image_summaries[1]}")
print(f"Summary 3: {image_summaries[2]}")

Summary 1:  The image is a table that shows the cost per credit, number of credits, total fee, and level fees for various courses. The cost per credit is in rupees, and the number of credits is listed next to it. The total fee is calculated by multiplying the cost per credit by the number of credits. The level fees are listed in the last column.
Summary 2:  The image is a table that shows the number of credits required for each level of study in a particular program. The levels are Foundation, Diploma in Data Science (DS), Diploma in Pharmacy (P), Bachelor of Science (BSc), and Bachelor of Studies (BS). For each level, the number of credits required for theory, project, apprenticeship, NPTEL, and MG/HS/HM are listed.
Summary 3:  The image describes the course structure of a Bachelor of Science in Data Science and Application program at the Indian Institute of Technology Madras. The program consists of three levels: Foundation, Diploma, and Degree. The Foundation level has 8 courses and

# Step 5: Build Multi-Vector Retrieval

### What is a multi-vector retriever?

Alright, let's make this fun and simple!

Imagine you have a big box of toys, but the toys are all mixed up. Some are action figures, some are puzzle pieces, and others are crayons. One day, your friend asks, "Can you find me something to draw with?" and another asks, "Can you find me a toy to play with?"

A **multi-vector retriever** is like a super-smart robot helper. It doesn't just search for one type of toy (like crayons) but knows how to look for all kinds of toys (action figures, puzzle pieces, and crayons) at the same time. It organizes the toys into groups based on what they’re good for, and then, when someone asks for a specific kind of toy, it quickly fetches the best match!

### Example in Action
Say you have:
- **Text**: A storybook
- **Pictures**: A drawing of a dinosaur
- **Tables**: A list of your favorite snacks

If someone asks, "What’s the scariest dinosaur you’ve seen?", the multi-vector retriever would:
1. Look in the storybook for mentions of "dinosaurs."
2. Search the drawing for a picture of a dinosaur.
3. Quickly fetch that information, so you can answer: “A scary T-Rex!”

So, it’s like having a helper who can find answers no matter what form the information is in—text, pictures, or lists. Cool, right? 😊

In [ ]:
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_google_vertexai import VertexAIEmbeddings

In [ ]:
def create_multi_vector_retriever(
   vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
   """
   Create retriever that indexes summaries, but returns raw images or texts.
   """
   store = InMemoryStore()
   id_key = "doc_id"

   retriever = MultiVectorRetriever(
       vectorstore=vectorstore,
       docstore=store,
       id_key=id_key,
   )

   def add_documents(retriever, doc_summaries, doc_contents):
       doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
       summary_docs = [
           Document(page_content=s, metadata={id_key: doc_ids[i]})
           for i, s in enumerate(doc_summaries)
       ]
       retriever.vectorstore.add_documents(summary_docs)
       retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

   if text_summaries:
       add_documents(retriever, text_summaries, texts)
   if table_summaries:
       add_documents(retriever, table_summaries, tables)
   if image_summaries:
       add_documents(retriever, image_summaries, images)

   return retriever

vectorstore = Chroma(
   collection_name="mm_rag_cj_blog",
   embedding_function=VertexAIEmbeddings(model_name="textembedding-gecko@latest"), # Think of this as the tool that turns your information into special "codes" (vectors) the digital bookshelf can understand. Here, it’s using VertexAIEmbeddings, a model designed to create these codes based on the latest technology (textembedding-gecko@latest).
)
"""
Imagine Chroma as a digital bookshelf that stores summaries or information in a way that makes it super easy to search later. Each piece of information (like text or image summaries) is turned into a "vector" (a mathematical representation) so the system can find things quickly.
"""

retriever_multi_vector_img = create_multi_vector_retriever(
   vectorstore,
   text_summaries,
   texts,
   table_summaries,
   tables,
   image_summaries,
   img_base64_list,
)

<ipython-input-18-058bfde9a57d>:34: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


The function **`create_multi_vector_retriever`** is like a librarian who organizes summaries of books, images, and tables in a library but gives you the actual book or image whenyou ask for it.

### Explanation:
- **Indexing summaries**: Imagine writing a short note about each book in the library to remember what it’s about. This is like creating summaries for texts, tables, or images.
- **Returning raw content**: But when someone wants to borrow the book (or image), the librarian gives them the full book, not just the note.

### Example:
Imagine you have:
1. A **storybook** with the summary: *"A magical adventure with a brave wizard."*
2. A **table** with the summary: *"A list of magical creatures."*
3. An **image** of a dragon, with the summary: *"A fierce red dragon flying over mountains."*

Here’s how it works:
1. All these summaries are stored in a database so that they’re easy to search.
2. If you ask for "Tell me about dragons," the retriever finds the dragon image’s summary based on its description.
3. Instead of just giving you the summary (*"A fierce red dragon flying over mountains"*), it retrieves and returns the **actual image of the dragon**.

In short, the retriever uses summaries to locate things but delivers the original content. This approach is helpful in multimodal systems where users interact with different kinds of data like text, images, or tables!

### Does vector store has actual columns?

Instead of thinking about the vectorstore as a database with actual "columns," it's better to think of it as:

- A search-optimized system that stores embeddings.

- It uses **metadata (like document IDs)** to connect to other storage systems (e.g., docstore), which hold summaries and raw content.

# Step 6: Building Multi-Modal RAG

In [ ]:
import io
import re
import base64
from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages import HumanMessage
from langchain.schema import SystemMessage
from langchain_google_vertexai import ChatVertexAI
from PIL import Image
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def plt_img_base64(img_base64):
    """This function takes a base64 encoded image string (img_base64) and displays it as an image within the Jupyter Notebook."""
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />' #creates an HTML <img> tag with the src attribute set to the base64 encoded image data.
    display(HTML(image_html))


def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    # define a dictionary image_signatures containing the characteristic byte sequences (signatures) found at the beginning of different image file formats (JPG, PNG, GIF, WEBP).
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def resize_base64_image(base64_string, size=(128, 128)):
    """
 This function takes a base64 encoded image string and resizes the image to the specified dimensions, returning the resized image as a base64 encoded string.

    """
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    resized_img = img.resize(size, Image.LANCZOS)

    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def split_image_text_types(docs):
    """
This function takes a list of documents (docs) and separates them into two categories: base64-encoded images and text content. It then returns a dictionary containing these separated elements.
"""
    b64_images = []
    texts = []
    for doc in docs:
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    if len(b64_images) > 0:
        return {"images": b64_images[:1], "texts": []}
    return {"images": b64_images, "texts": texts}


def img_prompt_func(data_dict):
    """
    This function takes a dictionary (data_dict) containing the context (texts and images) and the user's question. It formats this information into a prompt suitable for a multi-modal language model, specifically for the ChatVertexAI model.
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"]) # This line joins any text elements from the data_dict into a single string, separated by newlines.

    SYSTEM_PROMPT = """
      You are a professional support person of IITM BS Degree tasked with providing help.
      You will be given a mixture of text, tables, and image(s) usually of charts or graphs.
      Use this information to provide help related to the user question.
      Make sure your response doesn't contain any information outside the provided context.
      If the information is not available in the provided context, say "I'm sorry, but I couldn't find information about that in the document." Response should be in a markdown format in bullet points if needed.
      """
    messages = [] # Initializes an empty list to store prompt messages.

    text_message = {
        "type": "text",
        "text": (
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and/or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)

    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    return [SYSTEM_PROMPT, HumanMessage(content=messages)]

In [ ]:
def multi_modal_rag_chain(retriever):
    """
    This function constructs a multi-modal (RAG) chain. This chain orchestrates the process of retrieving relevant information from a knowledge base and using it to answer user questions, handling both text and images.
    """
    model = ChatVertexAI(
        temperature=0, #  Controls the randomness of the model's output (0 means deterministic).
        model_name="gemini-pro-vision", # Specifies the model to use, likely one capable of handling both text and images.
        max_output_tokens=1024, # max_output_tokens=1024: Limits the length of the model's response.

    )

    chain = ( # Defines the RAG chain using a series of steps
        {
            "context": retriever | RunnableLambda(split_image_text_types), # This part handles retrieving relevant context from the knowledge base.
            #"retriever" is the retriever object (likely a MultiVectorRetriever) used to search the knowledge base.
            # RunnableLambda(split_image_text_types): This function is applied to the retrieved documents to separate images and texts.
            "question": RunnablePassthrough(), # This simply passes the user's question through to the next step.
        }
        | RunnableLambda(img_prompt_func) # This function takes the retrieved context and the question and formats them into a prompt for the language model.
        | model # This is the ChatVertexAI language model, which processes the prompt and generates an answer
        | StrOutputParser() # This parses the model's output into a string.
    )

    return chain

Imagine a librarian who can find information in books and images to answer your questions. This function is like creating a set of instructions for the librarian:

1. Find relevant information: The librarian uses a search tool (retriever) to locate books and images related to your question.
2. Organize the information: The librarian separates the text from the images to make it easier to understand.
3. Prepare a summary: The librarian summarizes the relevant information from the books and images in a way that's easy for you to understand.
4. Answer your question: The librarian uses their knowledge and the prepared summary to answer your question.

The multi_modal_rag_chain function does something similar, but with a language model instead of a librarian.

In [ ]:
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector_img)

In [ ]:
# ChatVertexAI?

# Step 7: Testing and Evaluation

In [ ]:
from IPython.display import Markdown as md
# query = "Explain the fee structure for each level of the program, including fee waivers and additional costs."
query = "What are the BSc and BS credit requirements in IITM BS Degree?"
# query = "Explain the fee structure for each level of the program, including fee waivers."

# docs = retriever_multi_vector_img.invoke(query, limit=1)

# print(f"Number of relevant documents retrieved: {len(docs)}")

# print(docs, end="\n")
# print(type(docs))
result = chain_multimodal_rag.invoke(query)
# md(result)
result

'- The number of credits required to graduate with the BSc is 114 while that for the BS is 142.\n- The structure of the program is as follows:\n  - Level 1: The Foundation level (32 credits)\n  - Level 2: Diploma level (54 credits)\n  - Level 3: BSc Degree level courses (28 credits)\n  - Level 4: BS Degree level courses (28 credits)'

In [ ]:
# result = chain_multimodal_rag.invoke(query)

# # from IPython.display import Markdown as md
# # md(result)
# result

# SIMPLE QUESTIONS

## **Q1. What are the BSc and BS credit requirements in IITM BS Degree?**

- The number of credits required to graduate with the BSc is 114 while that for the BS is 142.
- The structure of the program is as follows:
  - Level 1: The Foundation level (32 credits)
  - Level 2: Diploma level (54 credits)
  - Level 3: BSc Degree level courses (28 credits)
  - Level 4: BS Degree level courses (28 credits)

## **Q2. What are the different exit options available in the IIT Madras BS Degree program?** (CORRECT ANSWER)
 There are multiple exit points in the IIT Madras BS in Data Science & Applications program.
1. After completing the foundation level, you can exit with a Foundation Certificate from ITM CODE. **(ATTENTION: wtf is this ITM CODE 😆)**
2. After completing the diploma level, you can exit with a Diploma in Programming and/or a Diploma in Data Science from IIT Madras.
3. After completing the degree level (BSc), you can exit with a BSc Degree in Programming & Data Science from IIT Madras.
4. After completing the degree level (BS), you can exit with a BS Degree in Data Science and Application from IIT Madras.

## **Q3: What is the eligibility to apply for the Qualifier Exam?** (CORRECT)
To be eligible to apply for the Qualifier Exam, you must meet the following criteria:

1. Have studied Mathematics and English in Class 10.
2. Have passed Class 12 or equivalent, regardless of age or academic background.
3. School students who have appeared for their Class 11 final exams can apply irrespective of their group/stream/board.

## **Q4. How many credits are required to complete the BS degree in Data Science & Applications?** (INCORRECT)
 To complete the BS degree in Data Science & Applications, 128 credits are required.

## **In which courses are project required?** (!!!!!!!!!!!!)
 Projects are required in the following courses:
1. Diploma in Data Science
2. Diploma in Programming
3. B.S.

## **How does the credit system in the IIT Madras BS in Data Science & Applications program compare to the traditional IITM campus credit system?** **(ATTENTION: Hallucinated answer in both cases whether you use summary or not...very bad)**
 The credit system in the IIT Madras BS in Data Science & Applications program is different from the traditional IITM campus credit system. In the BS in Data Science & Applications program, students are required to complete 120 credits to earn a degree. These credits are distributed across four levels of study: Foundation Level (32 credits), Diploma Level (54 credits), Degree Level (28 credits), and Degree Level (28 credits).

In the traditional IITM campus credit system, students are required to complete 180 credits to earn a degree. These credits are distributed across eight semesters of study.

One of the key differences between the two credit systems is the number of credits required to complete a degree. In the BS in Data Science & Applications program, students are required to complete 120 credits, while in the traditional IITM campus credit system, students are required to complete 180 credits. This means that students in the BS in Data Science & Applications program will have to take fewer courses to earn a degree.

Another key difference between the two credit systems is the distribution of credits across the different levels of study. In the BS in Data Science & Applications program, students are required to complete 32 credits at the Foundation Level, 54 credits at the Diploma Level, 28 credits at the Degree Level, and 28 credits at the Degree Level. In the traditional IITM campus credit system, students are required to complete 22 credits in the first year, 25 credits in the second year, 27 credits in the third year, and 26 credits in the fourth year. This means that students in the BS in Data Science & Applications program will have more flexibility in choosing their courses.

## **How can I get a loan for this program? Please give any contact information that can be useful.**

For any help with the SBI Loan Process, please contact Martina/Angelin at 9444020900/8608076093.


## **Explain the fee structure for each level of the program, including fee waivers and additional costs.** (Very complex answer in both cases, moreover there is NO mention of level 5 anywhere in the PDF, so totally incorrect answer)

 The fee structure for the IITM BS Degree program is as follows:

Foundation Level: Rs. 1000 per credit, with a total of 32 credits required. This amounts to a total fee of Rs. 32,000.

Diploma Level: Rs. 2500 per credit for theory courses and Rs. 10,000 for the project, with a total of 46 credits required (23x2=46 credits). This amounts to a total fee of Rs. 125,000.

BSc Level 3: Rs. 2500 per credit for mandatory courses and Rs. 1000 per credit for NPTEL courses, with a total of 20 credits required (5 mandatory + 15 NPTEL). This amounts to a total fee of Rs. 64,000 (40000+20000+4000).

BSc Level 4: Rs. 5000 per credit for Level 4 courses and Rs. 1000 per credit for NPTEL courses, with a total of 28 credits required (4 Level 4 + 4 Level 3 + 8 Level 4 + 8 Level 3 + 4 NPTEL). This amounts to a total fee of Rs. 70,000 (20000+32000+18000).

BSc Level 5: Rs. 5000 per credit for Level 5 courses and Rs. 1000 per credit for NPTEL courses, with a total of 28 credits required (4 Level 5 + 4 Level 4 + 8 Level 5 + 8 Level 4 + 4 NPTEL). This amounts to a total fee of Rs. 90,000 (20000+32000+38000).

BSc Level 6: Rs. 5000 per credit for Level 6 courses and Rs. 1000 per credit for NPTEL courses, with a total of 24 credits required (4 Level 6 + 4 Level 5 + 8 Level 6 + 8 Level 5 + 4 NPTEL). This amounts to a total fee of Rs. 80,000 (20000+32000+28000).

In addition to the tuition fees, there are also a number of other fees that students may be required to pay, such as a registration fee, a library fee, and a technology fee. These fees vary depending on the level of study and the specific program.

Students who are eligible for financial aid may be able to receive a fee waiver or reduction. For more information on financial aid, please visit the IITM BS Degree program website.